In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *

In [2]:
engine = create_engine("sqlite:///data/movies-db.sqlite")
conn = engine.connect()

In [3]:
df = pd.read_csv("data/genome-scores.csv")
df2 = pd.read_csv("data/genome-tags.csv")
df3 = pd.read_csv("data/links.csv")
df4 = pd.read_csv("data/movies.csv")
df5 = pd.read_csv("data/ratings.csv")
genreMatrix = pd.read_csv('clean_data/genreMatrix.csv')
links = pd.read_csv('links.csv')
other_info = pd.read_csv('clean_data/imdb_details_clean.csv')

In [4]:
genreMatrix_id = genreMatrix.rename(columns={'imdbID':'imdbID_formatted'})

In [5]:
tags_df = df.pivot(index='movieId', columns='tagId', values='relevance')

In [6]:
ratings_df = df5.groupby('movieId').mean()
ratings = ratings_df.drop(columns = ['userId', 'timestamp'])

In [7]:
movies_df = pd.merge(links, df4, how='inner', on='movieId')

In [8]:
movies_ratings_df = pd.merge(movies_df, ratings, how='inner', on='movieId')

In [9]:
movies_tags_ratings = pd.merge(movies_ratings_df, tags_df, how='inner', on='movieId')
movies_tags_ratings.to_sql('movies_tags', con=conn)

In [10]:
genreMatrix_ids = pd.merge(links, genreMatrix_id, how='inner', on='imdbID_formatted')

In [11]:
movies_genres_ratings = pd.merge(movies_ratings_df, genreMatrix_ids, how='inner', on='movieId')
movies_genres_ratings = movies_genres_ratings.drop(columns=['genres','imdbId_y','tmdbId_y','imdbID_formatted_y'])
movies_genres_ratings = movies_genres_ratings.rename(columns={'imdbId_x':'imdbId','tmdbId_x':'tmdbId','imdbID_formatted_x':'imdbID_formatted'})

In [44]:
rated = pd.get_dummies(other_info['Rated'])
rated['movieId'] = movies_genres_ratings['movieId']

In [48]:
production = pd.get_dummies(other_info['Production'])
production['movieId'] = movies_genres_ratings['movieId']

In [45]:
movies_more = pd.merge(movies_genres_ratings, rated, how='inner', on='movieId')

In [66]:
other_info['Plot']

0        A cowboy doll is profoundly threatened and jea...
1        When two kids find and play a magical board ga...
2        John and Max resolve to save their beloved bai...
3        Based on Terry McMillan's novel, this film fol...
4        George Banks must deal not only with the pregn...
                               ...                        
58145    A ticket scalper wants to make one last big de...
58146                                                  NaN
58147                                                  NaN
58148                                                  NaN
58149    Born into the seedy, underground world of pros...
Name: Plot, Length: 58150, dtype: object

In [51]:
movies_more2.to_sql('movies_genres2',con=conn)

OperationalError: (sqlite3.OperationalError) duplicate column name: Approved
[SQL: 
CREATE TABLE movies_genres2 (
	"index" BIGINT, 
	"movieId" BIGINT, 
	"imdbId" BIGINT, 
	"tmdbId" FLOAT, 
	"imdbID_formatted" TEXT, 
	title TEXT, 
	rating FLOAT, 
	"Action" BIGINT, 
	"Adult" BIGINT, 
	"Adventure" BIGINT, 
	"Animation" BIGINT, 
	"Biography" BIGINT, 
	"Comedy" BIGINT, 
	"Crime" BIGINT, 
	"Documentary" BIGINT, 
	"Drama" BIGINT, 
	"Family" BIGINT, 
	"Fantasy" BIGINT, 
	"Film-Noir" BIGINT, 
	"Game-Show" BIGINT, 
	"History" BIGINT, 
	"Horror" BIGINT, 
	"Music" BIGINT, 
	"Musical" BIGINT, 
	"Mystery" BIGINT, 
	"News" BIGINT, 
	"Reality-TV" BIGINT, 
	"Romance" BIGINT, 
	"Sci-Fi" BIGINT, 
	"Short" BIGINT, 
	"Sport" BIGINT, 
	"Talk-Show" BIGINT, 
	"Thriller" BIGINT, 
	"War" BIGINT, 
	"Western" BIGINT, 
	"(Banned)" BIGINT, 
	"AO" BIGINT, 
	"APPROVED" BIGINT, 
	"Approved" BIGINT, 
	"C" BIGINT, 
	"E" BIGINT, 
	"G" BIGINT, 
	"GP" BIGINT, 
	"K-13" BIGINT, 
	"M" BIGINT, 
	"M/PG" BIGINT, 
	"NC-17" BIGINT